In [2]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
cmap = plt.get_cmap("inferno")

plt.figure(figsize=(10,10))
genres = "blues classical country disco hiphop jazz metal pop reggae rock".split()

for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
        plt.axis('off')
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [4]:
header = "filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate"
for i in range(1, 21):
    header += f' mfcc{i}'
header += " label"
header = header.split()

In [8]:
file = open("data.csv", "w", newline="")
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'  
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open("data.csv", "a", newline="")
        with file:
            write = csv.writer(file)
            write.writerow(to_append.split())

In [9]:
data = pd.read_csv("data.csv")
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [10]:
data = data.drop(["filename"], axis=1)

In [11]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
len(y_train)

800

In [16]:
len(y_test)

200

In [17]:
X_train[10]

array([ 0.14525217, -0.01109839, -0.04881127,  0.06671925,  0.14159022,
       -0.03016868,  0.50317882,  0.30724746, -0.13507282,  0.31702482,
       -1.50034247,  0.77767463, -1.49029305,  0.980723  , -1.24281638,
        1.11633256, -0.61341417,  0.7331609 , -0.85223579,  1.20386402,
       -2.3036464 ,  0.46541828, -1.42923099,  1.82023776, -0.85875807,
        0.86797148])

In [18]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation="relu"))

model.add(layers.Dense(10, activation="softmax"))

In [19]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
history = model.fit(X_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
800/800 [==============================] - 0s 439us/step - loss: 2.1614 - acc: 0.2287
Epoch 2/20
800/800 [==============================] - 0s 17us/step - loss: 1.7892 - acc: 0.3912
Epoch 3/20
800/800 [==============================] - 0s 19us/step - loss: 1.5705 - acc: 0.4388
Epoch 4/20
800/800 [==============================] - 0s 17us/step - loss: 1.4040 - acc: 0.5038
Epoch 5/20
800/800 [==============================] - 0s 17us/step - loss: 1.2771 - acc: 0.5687
Epoch 6/20
800/800 [==============================] - 0s 17us/step - loss: 1.1834 - acc: 0.6075
Epoch 7/20
800/800 [==============================] - 0s 19us/step - loss: 1.1067 - acc: 0.6438
Epoch 8/20
800/800 [==============================] - 0s 17us/step - loss: 1.0402 - acc: 0.6750
Epoch 9/20
800/800 [==============================] - 0s 19us/step - loss: 0.9915 - acc: 0.6825
Epoch 10/20
800/800 [==============================] - 0s 17us/step - loss: 0.9350 - acc: 0.7137
Epoch 11/20
800/800 [=================

In [21]:
test_loss, test_acc = model.evaluate(X_test, y_test)

200/200 [==============================] - 0s 205us/step


In [22]:
print("test_acc: ", test_acc)

test_acc:  0.61


In [23]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [24]:
model = models.Sequential()

model.add(layers.Dense(512, activation="relu", input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(partial_x_train, partial_y_train, epochs=30, batch_size=512, validation_data=(x_val, y_val))

results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 687us/step - loss: 2.2897 - acc: 0.1333 - val_loss: 2.1318 - val_acc: 0.3400
Epoch 2/30
600/600 [==============================] - 0s 33us/step - loss: 2.1020 - acc: 0.3567 - val_loss: 1.9918 - val_acc: 0.3500
Epoch 3/30
600/600 [==============================] - 0s 32us/step - loss: 1.9578 - acc: 0.3650 - val_loss: 1.8679 - val_acc: 0.3700
Epoch 4/30
600/600 [==============================] - 0s 32us/step - loss: 1.8321 - acc: 0.3800 - val_loss: 1.7399 - val_acc: 0.4100
Epoch 5/30
600/600 [==============================] - 0s 33us/step - loss: 1.6991 - acc: 0.4050 - val_loss: 1.6163 - val_acc: 0.4300
Epoch 6/30
600/600 [==============================] - 0s 32us/step - loss: 1.5694 - acc: 0.4583 - val_loss: 1.5076 - val_acc: 0.4650
Epoch 7/30
600/600 [==============================] - 0s 33us/step - loss: 1.4394 - acc: 0.5433 - val_loss: 1.4282 - val_acc: 0.5300
Epoch 8/30
600/600 [==

In [25]:
results

[1.1779918813705443, 0.59]

In [26]:
predictions = model.predict(X_test)

In [27]:
predictions[0].shape

(10,)

In [28]:
np.sum(predictions[0])

1.0

In [29]:
np.argmax(predictions[0])

7